<a href="https://colab.research.google.com/github/aashutoshkumarbhardwaj/Rakshak-2.0/blob/main/7inone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smart Ambulance — All-in-one Colab Notebook
**Build & run all 7 components in one notebook**  
Components:
1. YOLO detector (injury, face, chest, bleeding, unconscious)
2. Blood segmentation (U-Net prototype)
3. Pain classifier (Face -> pain score)
4. Breathing estimator (optical flow)
5. Remote PPG (CHROM method)
6. Condition classifier (XGBoost)
7. LLM-style report generator (T5-small fine-tune / template fallback)

**Usage:** run cells sequentially. Upload your datasets to `/content/data/` or mount Google Drive.


In [8]:
# Choose GPU runtime in Colab before running this cell.

import sys
import subprocess

print("Step 1: Upgrading pip...")
!pip install --upgrade pip setuptools wheel

print("\nStep 2: Installing PyTorch (This is large, please wait)...")
# Removed -q to show progress. Added --no-cache-dir to prevent memory issues.
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --no-cache-dir

print("\nStep 3: Installing Computer Vision & ML Libraries...")
# Removed >/dev/null so you can see dependency resolution
!pip install ultralytics==8.4.8 opencv-python-headless==4.8.0.74
!pip install segmentation-models-pytorch==0.3.0 albumentations==1.3.1 timm==0.9.2

print("\nStep 4: Installing Utilities and Transformers...")
!pip install xgboost scikit-learn pandas matplotlib transformers accelerate peft datasets sentencepiece safetensors

print("\nStep 5: Installing MediaPipe...")
!pip install mediapipe==0.10.0

# Confirm installation
print("\nInstallation Complete.")
print("Python version:", sys.version)

Step 1: Upgrading pip...

Step 2: Installing PyTorch (This is large, please wait)...
Looking in indexes: https://download.pytorch.org/whl/cu121

Step 3: Installing Computer Vision & ML Libraries...
ERROR: Ignored the following yanked versions: 8.0.129, 8.0.174, 8.0.177, 8.1.21, 8.1.31, 8.2.7, 8.2.47
ERROR: Ignored the following versions that require a different python version: 8.0.10 Requires-Python >=3.7,<=3.11; 8.0.11 Requires-Python >=3.7,<=3.11; 8.0.12 Requires-Python >=3.7,<=3.11; 8.0.13 Requires-Python >=3.7,<=3.11; 8.0.14 Requires-Python >=3.7,<=3.11; 8.0.15 Requires-Python >=3.7,<=3.11; 8.0.16 Requires-Python >=3.7,<=3.11; 8.0.17 Requires-Python >=3.7,<=3.11; 8.0.18 Requires-Python >=3.7,<=3.11; 8.0.19 Requires-Python >=3.7,<=3.11; 8.0.20 Requires-Python >=3.7,<=3.11; 8.0.21 Requires-Python >=3.7,<=3.11; 8.0.22 Requires-Python >=3.7,<=3.11; 8.0.23 Requires-Python >=3.7,<=3.11; 8.0.24 Requires-Python >=3.7,<=3.11; 8.0.25 Requires-Python >=3.7,<=3.11; 8.0.26 Requires-Python >=3.7

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# Create workspace dir
!mkdir -p /content/smart_ambulance
%cd /content/smart_ambulance
!mkdir -p data weights notebooks outputs


Mounted at /content/drive
/content/smart_ambulance


In [4]:
# utils: saving, loading, image helpers
import os, json, time
from pathlib import Path
import cv2, numpy as np
from IPython.display import display, Image

def save_json(obj, path):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2)

def load_json(path):
    with open(path) as f:
        return json.load(f)

def show_image(img_bgr, scale=1.0):
    h,w = img_bgr.shape[:2]
    _,buf = cv2.imencode('.jpg', img_bgr)
    display(Image(data=buf.tobytes()))


In [5]:
# Data layout instructions (prepare these in /content/smart_ambulance/data)
# data/
#   yolo/
#     images/train/*.jpg
#     images/val/*.jpg
#     labels/train/*.txt  # YOLO format
#     labels/val/*.txt
#     data.yaml           # with paths and class names
#   seg/
#     images/*.png
#     masks/*.png
#   face_pain/
#     train/*.jpg, labels.csv (image,pain_score)
#   rppg/ (optional)
#
# If you don't have data yet, you can create synthetic or small sample sets to test code.
DATA_DIR = Path("/content/smart_ambulance/data")
YOLO_DIR = DATA_DIR / "yolo"
SEG_DIR = DATA_DIR / "seg"
FACE_DIR = DATA_DIR / "face_pain"
for p in [DATA_DIR, YOLO_DIR, SEG_DIR, FACE_DIR]:
    p.mkdir(parents=True, exist_ok=True)
print("Created data folders under", DATA_DIR)


Created data folders under /content/smart_ambulance/data


In [6]:
# Example YOLOv8 data.yaml (edit classes and paths per your dataset)
yolo_yaml = {
    "train": str(YOLO_DIR / "images" / "train"),
    "val": str(YOLO_DIR / "images" / "val"),
    "names": ["person","face","chest","injury","bleeding_moderate","bleeding_severe","unconscious"]
}
(YOLO_DIR / "data.yaml").write_text(json.dumps(yolo_yaml, indent=2))
print("Wrote", YOLO_DIR / "data.yaml")
print((YOLO_DIR / "data.yaml").read_text())


Wrote /content/smart_ambulance/data/yolo/data.yaml
{
  "train": "/content/smart_ambulance/data/yolo/images/train",
  "val": "/content/smart_ambulance/data/yolo/images/val",
  "names": [
    "person",
    "face",
    "chest",
    "injury",
    "bleeding_moderate",
    "bleeding_severe",
    "unconscious"
  ]
}


In [2]:
# Install Ultralytics explicitly
!pip install ultralytics

# Verify it works immediately
import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ultralytics]
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics version: 8.3.233


In [4]:
# Train YOLOv8 - adjust epochs, imgsz, and batch for Colab GPU
# Only run if dataset is prepared.
from pathlib import Path
DATA_YAML = str(YOLO_DIR / "data.yaml")
if Path(DATA_YAML).exists():
    print("Starting YOLO training (this may take a while)...")
    # example using ultralytics API
    from ultralytics import YOLO
    model = YOLO("yolov8n.pt")  # small model
    model.train(data=DATA_YAML, epochs=30, imgsz=640, batch=16, name="yolo_injury")
    # Save to weights folder
    !mkdir -p weights/yolo
    !cp -v runs/detect/yolo_injury/weights/best.pt weights/yolo/yolo_injury_best.pt
else:
    print("YOLO data.yaml not found; skip training. Place dataset at", YOLO_DIR)


NameError: name 'YOLO_DIR' is not defined